### Rolling Average in SQL

The following script will use a generated dataframe of stock prices to demonstrate the SQL and Pandas syntax to create a 4 week rolling average.

In [ ]:
import pandas as pd
import numpy as np
import pandasql as psql
from datetime import timedelta, date

In [ ]:
# create a dummy df showing the weekly stock prices for 3 random stock symbols on the ASX:

start_date = date(2024, 6, 1)
end_date = start_date + timedelta(weeks=9)  # 10 weeks from the start date
dates = pd.date_range(start=start_date, end=end_date, freq='W-FRI')

stock_symbols = ['BHP', 'CBA', 'WES']

data = {
    'stock_symbol': np.repeat(stock_symbols, len(dates)),
    'Date': np.tile(dates, len(stock_symbols)),
    'Price': np.round(np.random.uniform(35, 55, len(stock_symbols) * len(dates)), 2)
}
df = pd.DataFrame(data)


In [ ]:
# below is a demonstration of the syntax required to create a rolling 4wk average using SQL.
# note that the SQL used by pandasql (SQLite) does not support this syntax and will error out - the below is for a demonstration only.

query = """
SELECT stock_symbol
, Date
, Price
, AVG(Price) over (Partition by stock_symbol order by Date rows between 4 PRECEEDING AND CURRENT  ROW) as rolling_4wk_avg
FROM df
"""
result = psql.sqldf(query, locals())
print(result)

In [ ]:
# Calculate the rolling 4-week average using pandas:
df['rolling_4wk_avg'] = df.groupby('stock_symbol')['Price'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())

# Example of converting the entire column to floats:
df['rolling_4wk_avg'] = np.round(df['rolling_4wk_avg'].astype(float), 2)

df